In [1]:
import gym
import numpy as np 
import random

In [2]:
# create env 
env = gym.make('CartPole-v0')

In [3]:
env.reset()  # takes game to the initial state also return the intital state 

array([ 0.0118315 ,  0.01325219,  0.02903985, -0.03485435], dtype=float32)

In [4]:
for e in range(20): 
    obser = env.reset()
    for t in range(100):
        env.render()
        action = env.action_space.sample()
        state,reward,done,_ = env.step(action)
        if done : 
            print('Game Episode : {}/{} High Score {}'.format(e,20,t))
            break
        

env.close()
print('All epi over')

Game Episode : 0/20 High Score 19
Game Episode : 1/20 High Score 12
Game Episode : 2/20 High Score 12
Game Episode : 3/20 High Score 23
Game Episode : 4/20 High Score 25
Game Episode : 5/20 High Score 12
Game Episode : 6/20 High Score 31
Game Episode : 7/20 High Score 17
Game Episode : 8/20 High Score 11
Game Episode : 9/20 High Score 48
Game Episode : 10/20 High Score 18
Game Episode : 11/20 High Score 41
Game Episode : 12/20 High Score 8
Game Episode : 13/20 High Score 40
Game Episode : 14/20 High Score 9
Game Episode : 15/20 High Score 22
Game Episode : 16/20 High Score 12
Game Episode : 17/20 High Score 31
Game Episode : 18/20 High Score 31
Game Episode : 19/20 High Score 12
All epi over


In [5]:
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from collections import deque


In [6]:
class agent: 
    def __init__(self, _state_size, _action_size):
        self.state_size = _state_size
        self.action_size = _action_size   
        
        self.gamma = 0.95   # penalise future reward 
        self.epsilon = 1    #exploration to exploitation ratio at start we wanr the algorithm to be random  
        self.epsilon_min = 0.01
        
        self.epsilon_decay = 0.995  # with greater number of epochs the environment will use more and more of exploitation 
        self.memory = deque(maxlen = 2000)
        self.learning_rate = 1e-3
        
        self.model = self.create_model()
        
    def create_model(self):
        model = Sequential()
        model.add(Dense(24,input_dim = self.state_size, activation = 'relu'))
        model.add(Dense(24,activation = 'relu'))
        model.add(Dense(self.action_size, activation = 'linear'))
        model.compile(loss = 'mse', optimizer = Adam(learning_rate = self.learning_rate))
        return model
    
    def buffer(self,state,action,reward,next_state,done):
        self.memory.append((state,action,reward,next_state,done))
    def act(self,state):
        # act done acc to greedy Epsilon method 
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        else:
            return np.argmax(self.model.predict(state)[0])
    def train(self,batch_size = 32):
        minibatch = random.sample(self.memory,batch_size)
        for expe in minibatch:
            state,action,reward,next_state,done = expe
            if not done: 
                target = reward + self.gamma*np.amax(self.model.predict(next_state)[0])
            else:
                target = reward 
            target_f = self.model.predict(state)
            target_f[0][action] = target
            
            self.model.fit(state,target_f,epochs = 1, verbose = 0 )
        if self.epsilon > self.epsilon_min :
            self.epsilon *= self.epsilon_decay
    def load(self,name):
        self.model.load_weights(name)
    def save(self,name):
        self.model.save_weights(name)
    

In [7]:
n_epi = 500
state_size = 4 
action_size = 2 
ag = agent(_state_size=state_size,_action_size=action_size)
done = False

In [10]:
output_file = 'atari/'

First was trained for n_epi episodes then for another 70 episodes usins epsilon greedy policy 

In [13]:
for e in range(20):
    state = env.reset()
    state = state.reshape((1,state_size))
    for t in range(500):
        env.render()
        ac = ag.act(state)
        next_state,reward,done,_ = env.step(ac)
        reward = reward if not done else -10
        next_state = next_state.reshape((1,state_size))
        ag.buffer(state,ac,reward,next_state,done)
        state = next_state
        
        if done:
            print('Game Episode : {}/{} High Score {} Ecploration rate {} '.format(e,500,t,ag.epsilon))
            break
        batch_size = 32
    if len(ag.memory) > batch_size:
        ag.train(batch_size)
    if (e%5) == 0:
        continue
        ag.save(output_file + 'weigts_' + str(e) + '.hdf5')
        
print('Deep Q-learned modeled trained')
env.close()
    

Game Episode : 0/500 High Score 168 Ecploration rate 0.06380744126877576 
Game Episode : 1/500 High Score 199 Ecploration rate 0.06348840406243188 
Game Episode : 2/500 High Score 180 Ecploration rate 0.06317096204211972 
Game Episode : 3/500 High Score 175 Ecploration rate 0.06285510723190912 
Game Episode : 4/500 High Score 139 Ecploration rate 0.06254083169574957 
Game Episode : 5/500 High Score 132 Ecploration rate 0.062228127537270826 
Game Episode : 6/500 High Score 181 Ecploration rate 0.06191698689958447 
Game Episode : 7/500 High Score 173 Ecploration rate 0.061607401965086545 
Game Episode : 8/500 High Score 140 Ecploration rate 0.06129936495526111 
Game Episode : 9/500 High Score 176 Ecploration rate 0.0609928681304848 
Game Episode : 10/500 High Score 199 Ecploration rate 0.060687903789832374 
Game Episode : 11/500 High Score 199 Ecploration rate 0.06038446427088321 
Game Episode : 12/500 High Score 199 Ecploration rate 0.06008254194952879 
Game Episode : 13/500 High Score 

In [68]:
# checking the model 

In [17]:
state = env.reset()
state = state.reshape((1,4))
for t in range(199):
    env.render()
    ac = ag.act(state)
    state,reward,done,_ = env.step(ac)
    state = state.reshape((1,4))
    if done:
        
        print('Failed at {} '. format(t))
        break  
env.close()